In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd

import collections
import re
import pprint as pp
import numpy as np
import collections

import multiprocessing as mp
from multiprocessing.pool import ThreadPool

import math
import gzip
import pickle as pkl
from datetime import datetime
import matplotlib.pyplot as plt

import fonctions
import metric_loading

import itertools
from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join

import random
random.seed(0)


Using TensorFlow backend.


In [2]:
def get_behavior(directory,doc_name,init):
    
    all_features = ["text_tokens", "hashtags", "tweet_id", 
                    "present_media", "present_links", 
                    "present_domains", "tweet_type","language", 
                    "tweet_timestamp", "engaged_with_user_id",
                    "engaged_with_user_follower_count", "engaged_with_user_following_count", 
                    "engaged_with_user_is_verified", "engaged_with_user_account_creation",
                    "engaging_user_id", "engaging_user_follower_count", 
                    "engaging_user_following_count", "engaging_user_is_verified",
                    "engaging_user_account_creation", "engagee_follows_engager"]
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep='\x01', header=None)
    
    if init==True:
        labels = ['reply_timestamp','retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp']
        all_variables = all_features + labels
        df.columns = all_variables
        
        df['reply_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['reply_timestamp'] ]
        df['retweet_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['retweet_timestamp'] ]
        df['retweet_with_comment_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['retweet_with_comment_timestamp'] ]
        df['like_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['like_timestamp'] ]
        df['labels']=[ [x1,x2,x3,x4] for x1,x2,x3,x4 in zip(df.like_timestamp,df.retweet_timestamp,df.retweet_with_comment_timestamp,df.reply_timestamp) ]
        df = df.filter(['labels','engaging_user_id'], axis=1)
        
    return df

def label_to_idx(vec):
    if vec == [1,0,0,0]:
        return 0
    elif vec == [0,0,0,0]:
        return 1
    elif vec == [0,1,1,0]:
        return 2
    elif vec == [1,1,0,0]:
        return 3
    elif vec == [0,1,0,0]:
        return 4
    elif vec == [1,1,1,0]:
        return 5
    elif vec == [0,0,0,1]:
        return 6
    elif vec == [1,0,0,1]:
        return 7
    elif vec == [0,1,0,1]:
        return 8
    elif vec == [1,1,0,1]:
        return 9
    elif vec == [1,1,1,1]:
        return 10
    elif vec == [0,1,1,1]:
        return 11

def behavior_on_chunk(directory, chunk, chunk_id, init):
    
    engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
    
    user_presence = {}
    
    iteration=1
    for batch_file in chunk:

        df = get_behavior(directory, batch_file, init)
        df = df.groupby(['engaging_user_id']).last()
        dictio = reformater(df)
        
        select = { k:dictio[k] for k in np.unique(df.index) if k not in user_presence.keys()  }
        user_presence.update(select)
        
        reste = { k:dictio[k] for k in np.unique(df.index) if k in user_presence.keys()  }
        {update_reste(user_presence, key, v) for key,v in reste.items() }

        print(iteration)
        iteration=iteration+1
    
    print('cutting and saving...')
        
    if init==True:
        
        with gzip.open('/home/maxime/Desktop/RecSys2020/trends/2user_ratio_{}.pkl.gz'.format(chunk_id),'wb') as f:
            pkl.dump(user_presence,f)
        
    else:
                    
        with gzip.open('/home/maxime/Desktop/RecSys2020/trends/2update_user_ratio_{}.pkl.gz'.format(chunk_id),'wb') as f:
            pkl.dump(user_presence,f)
       
        return True
    
    return True

def reformater(df):
    dictio=collections.defaultdict(list)
    for labels,user in zip(df.labels,df.index):
        if user in dictio.keys():
            dictio[ user ][ label_to_idx(labels) ]+=1
        else:
            dictio[ user ]= [0]*12
            dictio[ user ][ label_to_idx(labels) ]+=1
            
    return dictio
    
def update_reste(user_presence, key,v):
    user_presence[key]=np.add(user_presence[key] , v)


In [6]:
%%time

batch_path='/home/maxime/Desktop/RecSys2020/data/batches'
batch_list = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]
chunks = fonctions.chunkIt(batch_list, 16)
directory = '/home/maxime/Desktop/RecSys2020/data/batches/'

if __name__ == '__main__':
    # Setup a list of processes that we want to run
        
    processes = [ mp.Process(target=behavior_on_chunk, args=(directory, chunk, chunk_id, True) ) for chunk_id, chunk in zip([0,1,2,3],[ chunks[0],chunks[1],chunks[2],chunks[3] ] ) ]

    # Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join() 

1
1
1
1
2
2
2
2
3
3
3
3
4
4
4
4
5
5
5
5
6
6
6
6
7
7
7
7
8
8
8
8
9
9
9
9
10
10
10
10
11
11
11
11
12
12
12
12
13
13
13
13
14
14
14
15
14
15
15
16
15
16
16
17
17
16
17
18
18
17
18
19
19
18
19
20
19
20
20
21
20
21
21
22
22
21
22
23
22
23
23
24
23
24
24
25
25
25
24
26
26
26
25
27
27
27
26
28
28
28
27
29
29
29
30
30
28
30
31
31
29
31
32
30
32
32
33
31
33
33
34
34
32
34
35
35
33
35
36
36
34
37
36
37
35
37
38
38
36
39
38
39
37
39
40
40
38
40
41
41
39
41
42
42
40
42
43
43
41
43
44
44
44
42
45
45
45
43
46
46
46
47
44
47
47
45
48
48
48
49
46
49
49
50
47
50
50
51
48
51
51
52
49
52
52
53
50
53
53
54
51
54
54
55
52
55
55
56
56
53
56
57
54
57
57
58
58
55
58
56
59
59
59
60
57
60
60
61
61
58
61
59
62
62
62
63
63
60
63
64
64
61
64
65
65
62
65
66
66
63
66
67
67
64
67
68
68
65
68
69
66
69
69
70
67
70
71
70
68
71
71
72
69
72
73
72
73
70
74
73
74
71
74
75
75
72
76
75
76
73
77
76
77
74
78
77
78
75
78
79
79
76
80
79
80
77
80
81
81
78
81
82
82
79
83
82
83
80
84
83
84
81
85
84
85
82
85
86
86
83
86
87
87
84
88
8

In [1]:
1873604/6561475

0.2855461614957003